<a href="https://colab.research.google.com/github/YashiPi/GNN-Bug-Triage-Assistant/blob/main/Bug_Triage_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.7 MB/s eta 0:00:00


In [ ]:
import csv
from github import Github
from google.colab import userdata

print("Libraries installed.")

Libraries installed.


In [ ]:
try:
    GITHUB_TOKEN = userdata.get('GITHUB_PAT')
    print("Successfully fetched GITHUB_PAT secret.")
except userdata.SecretNotFoundError:
    print("Secret 'GITHUB_PAT' not found.")
    print("Please add it using the 'key' icon (🔑) on the left panel.")
    raise

if not GITHUB_TOKEN:
    raise ValueError("GitHub PAT is empty. Please check your Colab Secret.")

Successfully fetched GITHUB_PAT secret.


In [ ]:
g = Github(GITHUB_TOKEN)

/tmp/ipython-input-1333792976.py:1: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(GITHUB_TOKEN)


In [ ]:
REPO_NAME = "microsoft/vscode"
print(f"Connecting to repository: {REPO_NAME}")

try:
    repo = g.get_repo(REPO_NAME)
except Exception as e:
    print(f"Error accessing repository: {e}")
    # This often happens if the token is invalid or doesn't have 'public_repo' scope
    raise

Connecting to repository: microsoft/vscode


In [ ]:
print("Fetching closed bug issues...")

output_filename = 'bugs.csv'

with open(output_filename, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["bug_id", "title", "reporter_username", "fixer_username"])

    # Get issues labeled 'bug' that are 'closed'
    issues = repo.get_issues(
        state='closed',
        labels=[repo.get_label("bug")]
    )

    # --- Limit for testing ---
    # 100 valid bugs to start.
    count = 0
    total_checked = 0
    BUG_LIMIT = 100

    for issue in issues:
        total_checked += 1

        # We only want bugs that were *fixed* (closed by a user)
        if issue.closed_by:
            bug_id = issue.number
            title = issue.title
            reporter_username = issue.user.login
            fixer_username = issue.closed_by.login

            # Write data to our CSV
            writer.writerow([bug_id, title, reporter_username, fixer_username])
            count += 1

            if count % 20 == 0:
                print(f"  Processed {total_checked} issues... Found {count} valid bugs...")

        if count >= BUG_LIMIT:
            print(f"Reached test limit of {BUG_LIMIT} bugs.")
            break

        # To avoid hitting rate limits too fast even when iterating
        if total_checked > BUG_LIMIT * 10:
             print(f"Checked {total_checked} issues without finding {BUG_LIMIT} bugs. Stopping to be safe.")
             break

print(f"Successfully created {output_filename} with {count} entries.")
print(f"You can see the file in the Colab 'Files' tab (folder icon 📁) on the left.")

Fetching closed bug issues...
  Processed 20 issues... Found 20 valid bugs...
  Processed 40 issues... Found 40 valid bugs...
  Processed 60 issues... Found 60 valid bugs...
  Processed 80 issues... Found 80 valid bugs...
  Processed 100 issues... Found 100 valid bugs...
Reached test limit of 100 bugs.
Successfully created bugs.csv with 100 entries.
You can see the file in the Colab 'Files' tab (folder icon 📁) on the left.


We now have the bugs.csv file <br>

The next, most important step is to find out which files were changed to fix each bug. This will build the core of our graph, connecting Bugs -> Files <- Developers.

In [ ]:
import csv
from github import Github
from google.colab import userdata
import time

print("--- Script 2: Link-Finder ---")

try:
    GITHUB_TOKEN = userdata.get('GITHUB_PAT')
    print("Successfully fetched GITHUB_PAT secret.")
except userdata.SecretNotFoundError:
    print("Secret 'GITHUB_PAT' not found. Please add it.")
    raise

g = Github(GITHUB_TOKEN, retry=5, timeout=15)
REPO_NAME = "microsoft/vscode"
print(f"Connecting to repository: {REPO_NAME}")
try:
    repo = g.get_repo(REPO_NAME)
except Exception as e:
    print(f"Error accessing repository: {e}")
    raise

--- Script 2: Link-Finder ---
Successfully fetched GITHUB_PAT secret.
Connecting to repository: microsoft/vscode


/tmp/ipython-input-2712398434.py:16: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  g = Github(GITHUB_TOKEN, retry=5, timeout=15)


In [ ]:
bug_files_csv = 'bug_to_files.csv'
dev_files_csv = 'dev_to_files.csv'

all_files = set()

with open('bugs.csv', 'r', encoding='utf-8') as f_bugs, \
     open(bug_files_csv, 'w', newline='', encoding='utf-8') as f_bug_files, \
     open(dev_files_csv, 'w', newline='', encoding='utf-8') as f_dev_files:

    bug_reader = csv.DictReader(f_bugs)
    bug_files_writer = csv.writer(f_bug_files)
    dev_files_writer = csv.writer(f_dev_files)

    bug_files_writer.writerow(["bug_id", "filepath"])
    dev_files_writer.writerow(["developer_username", "filepath"])

    print("Starting to process bugs from bugs.csv...")

    processed_count = 0
    total_bugs = 0
    for row in bug_reader:
        bug_id = row['bug_id']
        fixer = row['fixer_username']
        total_bugs += 1

        query = f"repo:{REPO_NAME} {bug_id} is:pr is:merged"

        try:
            time.sleep(1)

            search_results = g.search_issues(query, sort = 'updated')

            if search_results.totalCount > 0:
                pr_summary = search_results[0]

                pr = repo.get_pull(pr_summary.number)

                pr_author = pr.user.login
                pr_merger = pr.merged_by.login if pr.merged_by else ""

                pr_files = pr.get_files()

                found_files = False
                for file in pr_files:
                    filepath = file.filename

                    bug_files_writer.writerow([bug_id, filepath])
                    dev_files_writer.writerow([fixer, filepath])

                    all_files.add(filepath)
                    found_files = True

                if found_files:
                    print(f"  Processed bug #{bug_id}: Found PR #{pr.number} (author: {pr_author}) with {pr_files.totalCount} files.")
                    processed_count += 1
                else:
                    print(f"  Processed bug #{bug_id}: Found PR #{pr.number}, but it had 0 files. Skipping.")

            else:
                print(f"  Processed bug #{bug_id}: No merged PR found that 'mentions' it. Skipping.")

        except Exception as e:
            print(f"  ERROR processing bug #{bug_id}: {e}")

            if "API rate limit exceeded" in str(e):
                print("!!! RATE LIMIT HIT. Sleeping for 60 seconds... !!!")
                time.sleep(60)

    print(f"\n--- Process Complete ---")
    print(f"Processed {total_bugs} bugs from csv.")
    print(f"Found PRs and files for {processed_count} bugs.")
    print(f"Created {bug_files_csv} and {dev_files_csv}.")


Starting to process bugs from bugs.csv...
  Processed bug #276573: No merged PR found that 'mentions' it. Skipping.
  Processed bug #276548: Found PR #276552 (author: roblourens) with 1 files.
  Processed bug #276407: Found PR #276409 (author: JeffreyCA) with 4 files.
  Processed bug #276406: Found PR #276409 (author: JeffreyCA) with 4 files.
  Processed bug #276305: No merged PR found that 'mentions' it. Skipping.
  Processed bug #276253: Found PR #276517 (author: Tyriar) with 1 files.
  Processed bug #276246: Found PR #276278 (author: bpasero) with 1 files.
  Processed bug #276169: No merged PR found that 'mentions' it. Skipping.
  Processed bug #276167: Found PR #276194 (author: mjbvz) with 1 files.
  Processed bug #276165: No merged PR found that 'mentions' it. Skipping.
  Processed bug #276132: No merged PR found that 'mentions' it. Skipping.
  Processed bug #276119: Found PR #276118 (author: lszomoru) with 1 files.
  Processed bug #276058: No merged PR found that 'mentions' it. S

In [ ]:
files_csv = 'files.csv'
with open(files_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["file_id", "filepath"])

    for i, filepath in enumerate(all_files):
        writer.writerow([i, filepath])

print(f"Created {files_csv} with {len(all_files)} unique filepaths.")

Created files.csv with 92 unique filepaths.


## Phase 2: Graph Construction & Feature Engineering

In [ ]:
# Install PyTorch
!pip install torch -q

In [ ]:
import torch
TORCH_VERSION = torch.__version__
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-{TORCH_VERSION}.html -q

!pip install sentence-transformers -q

print("All ML libraries installed successfully.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00
All ML libraries installed successfully.


In [ ]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData
from sentence_transformers import SentenceTransformer

print("--- Phase 2: Graph Builder & Feature Engineering ---")

try:
    bugs_df = pd.read_csv('bugs.csv')
    files_df = pd.read_csv('files.csv')
    bug_files_df = pd.read_csv('bug_to_files.csv')
    dev_files_df = pd.read_csv('dev_to_files.csv')
    print("Successfully loaded all 4 CSV files.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure all CSV files are in the Colab environment.")
    raise


--- Phase 2: Graph Builder & Feature Engineering ---
Successfully loaded all 4 CSV files.


In [ ]:
files_df = files_df.sort_values(by='file_id').set_index('file_id')
file_path_to_id = {row['filepath']: index for index, row in files_df.iterrows()}
num_files = len(files_df)
print(f"Mapped {num_files} files.")

unique_devs = set(bugs_df['fixer_username']).union(set(dev_files_df['developer_username']))
dev_to_id = {name: i for i, name in enumerate(unique_devs)}
num_devs = len(dev_to_id)
print(f"Mapped {num_devs} unique developers.")

bugs_with_files = set(bug_files_df['bug_id'])
bugs_df = bugs_df[bugs_df['bug_id'].isin(bugs_with_files)]
bug_id_to_node_idx = {bug_id: i for i, bug_id in enumerate(bugs_df['bug_id'])}
num_bugs = len(bug_id_to_node_idx)
print(f"Mapped {num_bugs} bugs (that have file connections).")


Mapped 92 files.
Mapped 26 unique developers.
Mapped 62 bugs (that have file connections).


In [ ]:
print("Starting feature engineering...")

# (A) Bug Features: Use Sentence Transformer on bug titles
# This model is small, fast, and very effective
model = SentenceTransformer('all-MiniLM-L6-v2')
print("  Loaded SentenceTransformer model.")

# Get bug titles in the correct order
bug_titles = [bugs_df[bugs_df['bug_id'] == bug_id].iloc[0]['title'] for bug_id in bug_id_to_node_idx.keys()]

# Create embeddings.
with torch.no_grad():
    bug_features = model.encode(bug_titles, convert_to_tensor=True, show_progress_bar=True)
bug_features = bug_features.to(torch.float)
print(f"  Created bug features tensor with shape: {bug_features.shape}")

# (B) Developer Features: Simple identity matrix (one-hot)
# The GNN will learn features based on connections
dev_features = torch.eye(num_devs, dtype=torch.float)
print(f"  Created developer features tensor with shape: {dev_features.shape}")

# (C) File Features: Simple identity matrix
file_features = torch.eye(num_files, dtype=torch.float)
print(f"  Created file features tensor with shape: {file_features.shape}")


In [ ]:
print("Building HeteroData object...")
data = HeteroData()

data['bug'].x = bug_features
data['developer'].x = dev_features
data['file'].x = file_features

# (A) Edge: (Bug) - 'mentions' -> (File)
# We need to map the CSV rows to our new integer IDs
bug_indices = []
file_indices_for_bugs = []
for _, row in bug_files_df.iterrows():
    if row['bug_id'] in bug_id_to_node_idx and row['filepath'] in file_path_to_id:
        bug_indices.append(bug_id_to_node_idx[row['bug_id']])
        file_indices_for_bugs.append(file_path_to_id[row['filepath']])

bug_file_edge_index = torch.tensor([bug_indices, file_indices_for_bugs], dtype=torch.long)
data['bug', 'mentions', 'file'].edge_index = bug_file_edge_index
print(f"  Added {bug_file_edge_index.shape[1]} 'mentions' edges.")


# (B) Edge: (Developer) - 'modified' -> (File)
dev_indices = []
file_indices_for_devs = []
for _, row in dev_files_df.iterrows():
    if row['developer_username'] in dev_to_id and row['filepath'] in file_path_to_id:
        dev_indices.append(dev_to_id[row['developer_username']])
        file_indices_for_devs.append(file_path_to_id[row['filepath']])

dev_file_edge_index = torch.tensor([dev_indices, file_indices_for_devs], dtype=torch.long)
data['developer', 'modified', 'file'].edge_index = dev_file_edge_index
print(f"  Added {dev_file_edge_index.shape[1]} 'modified' edges.")

Building HeteroData object...
  Added 121 'mentions' edges.
  Added 121 'modified' edges.


In [ ]:
# This is our target: the (Developer, Bug) pair we want to predict
fixer_indices = []
bug_indices_for_fixer = []
for _, row in bugs_df.iterrows():
    if row['fixer_username'] in dev_to_id:
        fixer_indices.append(dev_to_id[row['fixer_username']])
        bug_indices_for_fixer.append(bug_id_to_node_idx[row['bug_id']])

data['developer', 'fixes', 'bug'].edge_label_index = torch.tensor([fixer_indices, bug_indices_for_fixer], dtype=torch.long)
print(f"  Added {len(fixer_indices)} 'fixes' edges as our ground truth.")

print("\n--- Graph Construction Complete! ---")
print(f"\nFinal HeteroData object:\n{data}")
print("\nThis object 'data' now holds our entire graph.")

torch.save(data, 'graph_data.pt')
print("\nGraph data saved to 'graph_data.pt'.")

  Added 62 'fixes' edges as our ground truth.

--- Graph Construction Complete! ---

Final HeteroData object:
HeteroData(
  bug={ x=[62, 384] },
  developer={ x=[26, 26] },
  file={ x=[92, 92] },
  (bug, mentions, file)={ edge_index=[2, 121] },
  (developer, modified, file)={ edge_index=[2, 121] },
  (developer, fixes, bug)={ edge_label_index=[2, 62] }
)

This object 'data' now holds our entire graph.

Graph data saved to 'graph_data.pt'.


## Phase 3: Model Building & Training

In [ ]:
import torch
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

print("--- Step 1: Loading, Fixing, and Splitting Data (v3) ---")

# Load saved graph
data = torch.load('graph_data.pt', weights_only = False)

print("Applying hotfix 1: Renaming 'edge_label_index' to 'edge_index' for 'fixes'...")
if 'edge_label_index' in data['developer', 'fixes', 'bug']:
    data['developer', 'fixes', 'bug'].edge_index = data['developer', 'fixes', 'bug'].edge_label_index
    del data['developer', 'fixes', 'bug'].edge_label_index
    print("Fix 1 applied successfully.")
else:
    print("Fix 1 already applied.")


# We must ONLY make the MESSAGE PASSING edges bidirectional.
# T.ToUndirected() on the whole graph was a mistake as it leaked the answer.
# We will add the reverse edges manually to allow message flow.
print("Applying hotfix 2: Manually reversing message-passing edges...")

edge_index_dev_file = data['developer', 'modified', 'file'].edge_index

data['file', 'rev_modified', 'developer'].edge_index = edge_index_dev_file.flip([0])

edge_index_bug_file = data['bug', 'mentions', 'file'].edge_index

data['file', 'rev_mentions', 'bug'].edge_index = edge_index_bug_file.flip([0])

print("Fix 2 applied. 'developer' and 'bug' nodes can now be updated.")

transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=True,
    edge_types=[('developer', 'fixes', 'bug')],
    rev_edge_types=[('bug', 'rev_fixes', 'developer')],
)

train_data, val_data, test_data = transform(data)

print("\n--- Data Splitting Complete ---")
print(f"Original Data (pre-split, after fixes):\n{data}")

print("\nTraining Data (what the model sees):")
print(train_data)
print("\nValidation Data (for checking):")
print(val_data)

--- Step 1: Loading, Fixing, and Splitting Data (v3) ---
Applying hotfix 1: Renaming 'edge_label_index' to 'edge_index' for 'fixes'...
Fix 1 applied successfully.
Applying hotfix 2: Manually reversing message-passing edges...
Fix 2 applied. 'developer' and 'bug' nodes can now be updated.

--- Data Splitting Complete ---
Original Data (pre-split, after fixes):
HeteroData(
  bug={ x=[62, 384] },
  developer={ x=[26, 26] },
  file={ x=[92, 92] },
  (bug, mentions, file)={ edge_index=[2, 121] },
  (developer, modified, file)={ edge_index=[2, 121] },
  (developer, fixes, bug)={ edge_index=[2, 62] },
  (file, rev_modified, developer)={ edge_index=[2, 121] },
  (file, rev_mentions, bug)={ edge_index=[2, 121] }
)

Training Data (what the model sees):
HeteroData(
  bug={ x=[62, 384] },
  developer={ x=[26, 26] },
  file={ x=[92, 92] },
  (bug, mentions, file)={ edge_index=[2, 121] },
  (developer, modified, file)={ edge_index=[2, 121] },
  (developer, fixes, bug)={
    edge_index=[2, 50],
    e

In [ ]:
# defining the simple GNN "blueprint"
# This is a standard 2-layer GraphSAGE model
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, out_channels)

        self.encoder = to_hetero(self.encoder, metadata, aggr='sum')

    def forward(self, x_dict, edge_index_dict):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return z_dict

    def decode(self, z_dict, edge_label_index):
        dev_emb = z_dict['developer']
        bug_emb = z_dict['bug']

        src_dev_nodes = edge_label_index[0]
        dst_bug_nodes = edge_label_index[1]

        dev_src_emb = dev_emb[src_dev_nodes]
        bug_dst_emb = bug_emb[dst_bug_nodes]

        # --- This is our prediction ---
        # We do a simple dot-product.
        # If the embeddings are "similar", the score will be high.
        pred = (dev_src_emb * bug_dst_emb).sum(dim=-1)
        return pred

print("--- Model Definition Complete ---")

print("Cleaning metadata for model initialization...")
node_types, edge_types = train_data.metadata()

clean_edge_types = []
for edge_type in edge_types:
    edge_name = edge_type[1]

    if 'fixes' not in edge_name:
        clean_edge_types.append(edge_type)

clean_metadata = (node_types, tuple(clean_edge_types))
print("Metadata cleaned.")

model = Model(
    hidden_channels=64,
    out_channels=64,
    metadata=clean_metadata
)
print(model)

--- Model Definition Complete ---
Cleaning metadata for model initialization...
Metadata cleaned.
Model(
  (encoder): GraphModule(
    (conv1): ModuleDict(
      (bug__mentions__file): SAGEConv((-1, -1), 64, aggr=mean)
      (developer__modified__file): SAGEConv((-1, -1), 64, aggr=mean)
      (file__rev_modified__developer): SAGEConv((-1, -1), 64, aggr=mean)
      (file__rev_mentions__bug): SAGEConv((-1, -1), 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (bug__mentions__file): SAGEConv((-1, -1), 64, aggr=mean)
      (developer__modified__file): SAGEConv((-1, -1), 64, aggr=mean)
      (file__rev_modified__developer): SAGEConv((-1, -1), 64, aggr=mean)
      (file__rev_mentions__bug): SAGEConv((-1, -1), 64, aggr=mean)
    )
  )
)


In [ ]:
import time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = model.to(device)
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# We use Binary Cross-Entropy with Logits Loss
# This is standard for binary (0/1) prediction
loss_fn = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    optimizer.zero_grad()

    z_dict = model.forward(train_data.x_dict, train_data.edge_index_dict)

    pred = model.decode(z_dict, train_data['developer', 'fixes', 'bug'].edge_label_index)

    target = train_data['developer', 'fixes', 'bug'].edge_label.to(torch.float)

    loss = loss_fn(pred, target)

    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test(data_to_test):
    model.eval()

    z_dict = model.forward(data_to_test.x_dict, data_to_test.edge_index_dict)

    pred = model.decode(z_dict, data_to_test['developer', 'fixes', 'bug'].edge_label_index)

    target = data_to_test['developer', 'fixes', 'bug'].edge_label

    pred = pred.cpu().numpy()
    target = target.cpu().numpy()

    return roc_auc_score(target, pred)

print("\n--- Starting Training ---")
start_time = time.time()

for epoch in range(1, 101):
    loss = train()

    if epoch % 10 == 0:
        val_auc = test(val_data)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}')

end_time = time.time()
print(f"\n--- Training Complete ---")
print(f"Total time: {end_time - start_time:.2f} seconds")


final_test_auc = test(test_data)

print(f'\n==============================')
print(f'   Final Test AUC: {final_test_auc:.4f}')
print(f'==============================')

Using device: cpu

--- Starting Training ---
Epoch: 010, Loss: 0.0302, Val AUC: 0.4722
Epoch: 020, Loss: 0.0001, Val AUC: 0.5833
Epoch: 030, Loss: 0.0000, Val AUC: 0.5833
Epoch: 040, Loss: 0.0000, Val AUC: 0.6111
Epoch: 050, Loss: 0.0000, Val AUC: 0.6111
Epoch: 060, Loss: 0.0000, Val AUC: 0.6111
Epoch: 070, Loss: 0.0000, Val AUC: 0.6111
Epoch: 080, Loss: 0.0000, Val AUC: 0.6111
Epoch: 090, Loss: 0.0000, Val AUC: 0.6111
Epoch: 100, Loss: 0.0000, Val AUC: 0.6111

--- Training Complete ---
Total time: 1.05 seconds

   Final Test AUC: 0.8333


In [ ]:
print("--- Verifying model keys before saving ---")

keys = model.state_dict().keys()

found_bad_keys = False
for k in keys:
    if "fixes" in k:
        print(f"FOUND BAD KEY: {k}")
        found_bad_keys = True

if found_bad_keys:
    print("\n--- ERROR! ---")
    print("Your 'model' object is still the old, bad one. Please restart runtime (Step 1) and try again.")
else:
    print("\n--- SUCCESS! ---")
    print("Model is correct. No 'fixes' layers found. Saving to file...")

    # Save the model
    model_save_path = 'gnn_bug_triage_model.pth'
    torch.save(model.state_dict(), model_save_path)

    print(f"Trained model weights saved to {model_save_path}")
    print("You can now download this new file.")

--- Verifying model keys before saving ---

--- SUCCESS! ---
Model is correct. No 'fixes' layers found. Saving to file...
Trained model weights saved to gnn_bug_triage_model.pth
You can now download this new file.


## Testing the model

In [ ]:
import pandas as pd
import torch
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
from sentence_transformers import SentenceTransformer
import os

GRAPH_PATH = "graph_data.pt"
MODEL_PATH = "gnn_bug_triage_model.pth"
BUGS_CSV_PATH = "bugs.csv"
DEV_FILES_CSV_PATH = "dev_to_files.csv"

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, metadata):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, out_channels)
        self.encoder = to_hetero(self.encoder, metadata, aggr='sum')
    def forward(self, x_dict, edge_index_dict):
        return self.encoder(x_dict, edge_index_dict)

print("--- Loading all assets for testing... ---")

device = torch.device('cpu')
full_data = torch.load(GRAPH_PATH, weights_only = False)

print("Adding reverse edges for message passing...")
edge_index_dev_file = full_data['developer', 'modified', 'file'].edge_index
full_data['file', 'rev_modified', 'developer'].edge_index = edge_index_dev_file.flip([0])
edge_index_bug_file = full_data['bug', 'mentions', 'file'].edge_index
full_data['file', 'rev_mentions', 'bug'].edge_index = edge_index_bug_file.flip([0])

print("Cleaning metadata for model initialization...")
node_types, edge_types = full_data.metadata()
clean_edge_types = []
for edge_type in edge_types:
    if 'fixes' not in edge_type[1]: # Filter out 'fixes' links
        clean_edge_types.append(edge_type)
clean_metadata = (node_types, tuple(clean_edge_types))
full_data = full_data.to(device)

print("Loading models...")
text_model = SentenceTransformer('all-MiniLM-L6-v2')
text_model.eval()

gnn_model = Model(hidden_channels=64, out_channels=64, metadata=clean_metadata)
gnn_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
gnn_model.eval()
print("Models loaded successfully.")

bugs_df = pd.read_csv(BUGS_CSV_PATH)
dev_files_df = pd.read_csv(DEV_FILES_CSV_PATH)
unique_devs = set(bugs_df['fixer_username']).union(set(dev_files_df['developer_username']))
dev_to_id = {name: i for i, name in enumerate(unique_devs)}
id_to_dev = {i: name for name, i in dev_to_id.items()}
print(f"Loaded {len(id_to_dev)} developer mappings.")

@torch.no_grad()
def predict_triage(bug_title, bug_body):
    print(f"\n--- Predicting for: '{bug_title}' ---")

    z_dict = gnn_model.forward(full_data.x_dict, full_data.edge_index_dict)

    new_bug_text = bug_title + " " + bug_body
    new_bug_raw_embedding = text_model.encode(new_bug_text, convert_to_tensor=True).to(device)
    new_bug_raw_embedding = new_bug_raw_embedding.to(torch.float)

    conv1_bug_layer = gnn_model.encoder.conv1['file__rev_mentions__bug']
    conv2_bug_layer = gnn_model.encoder.conv2['file__rev_mentions__bug']

    new_bug_h1 = conv1_bug_layer.lin_r(new_bug_raw_embedding).relu()
    new_bug_final_embedding = conv2_bug_layer.lin_r(new_bug_h1) # Shape is now [64]

    all_dev_embeddings = z_dict['developer'] # Shape [num_devs, 64]

    scores = (all_dev_embeddings * new_bug_final_embedding).sum(dim=-1)

    scores, top_k_indices = torch.topk(scores, 3)

    results = []
    for i in range(len(top_k_indices)):
        dev_node_id = top_k_indices[i].item()
        dev_username = id_to_dev.get(dev_node_id, "Unknown-Developer")
        score = scores[i].item()
        results.append({"username": dev_username, "score": float(score)})

    return results

# --- TEST IT! ---
recommendations = predict_triage(
    "Search bar is slow",
    "The main search function is performing poorly."
)

print("\nRECOMMENDATIONS:")
print(recommendations)

--- Loading all assets for testing... ---
Adding reverse edges for message passing...
Cleaning metadata for model initialization...
Loading models...
Models loaded successfully.
Loaded 26 developer mappings.

--- Predicting for: 'Search bar is slow' ---

RECOMMENDATIONS:
[{'username': 'jrieken', 'score': 1.0952004194259644}, {'username': 'lszomoru', 'score': 0.8538224697113037}, {'username': 'meganrogge', 'score': 0.8496090173721313}]


In [ ]:
# --- MORE TESTS ---

# Test 1: A bug about file management
print("--- Test Case 1: File Explorer Bug ---")
recommendations_file = predict_triage(
    "Cannot drag and drop file in sidebar",
    "The file explorer tree view doesn't update when I move a file. It's a rendering issue in the 'explorer' panel."
)
print("\nRECOMMENDATIONS (File Bug):")
print(recommendations_file)

# Test 2: A bug about UI/rendering
print("\n--- Test Case 2: UI Rendering Bug ---")
recommendations_ui = predict_triage(
    "Dropdown menu renders in wrong place",
    "The 'select' or 'dropdown' component in the workbench is not displaying correctly. It appears in the top corner of the screen."
)
print("\nRECOMMENDATIONS (UI Bug):")
print(recommendations_ui)

# Test 3: A generic bug about performance
print("\n--- Test Case 3: Performance Bug ---")
recommendations_perf = predict_triage(
    "App is slow on startup",
    "VSCode is taking a long time to load. Seems like a main process or 'workbench' loading issue."
)
print("\nRECOMMENDATIONS (Performance Bug):")
print(recommendations_perf)

--- Test Case 1: File Explorer Bug ---

--- Predicting for: 'Cannot drag and drop file in sidebar' ---

RECOMMENDATIONS (File Bug):
[{'username': 'Tyriar', 'score': 4.344862937927246}, {'username': 'meganrogge', 'score': 3.8368003368377686}, {'username': 'jrieken', 'score': 3.7280516624450684}]

--- Test Case 2: UI Rendering Bug ---

--- Predicting for: 'Dropdown menu renders in wrong place' ---

RECOMMENDATIONS (UI Bug):
[{'username': 'meganrogge', 'score': 2.5281147956848145}, {'username': 'jrieken', 'score': 2.506861686706543}, {'username': 'lszomoru', 'score': 1.188596248626709}]

--- Test Case 3: Performance Bug ---

--- Predicting for: 'App is slow on startup' ---

RECOMMENDATIONS (Performance Bug):
[{'username': 'benvillalobos', 'score': 0.7458015084266663}, {'username': 'jrieken', 'score': 0.7192777991294861}, {'username': 'lszomoru', 'score': 0.46238595247268677}]
